<a href="https://colab.research.google.com/github/Dats65lol/DESEMPE-O_ECONOMICO/blob/main/PIB_per_Capita_por_pais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
"""""DEFINITIVO"""""

"""LIBRERIAS"""

import requests  # repositorio de peticiones
import pandas as pd  # Para manipulación de datos en forma de DataFrame
import plotly.graph_objects as go  # Para crear gráficos interactivos con Plotly

"""CLASES"""

class IndicadorEconomico:
    """Representa un indicador económico como el PIB per cápita de un país en un año."""
    def __init__(self, nombre, pais, valor, year):  # Constructor que donde se ponen los atributos de la clase
        self.nombre = nombre  # Almacena el nombre del indicador
        self.pais = pais      # Almacena el código del país (ejemplo: 'COL')
        self.valor = valor    # Almacena el valor numérico del indicador
        self.year = year      # Almacena el año asociado al indicador

class AnalizadorEconomico:
    """Clase para almacenar y graficar indicadores económicos."""
    def __init__(self):  # Constructor que empieza la estructura interna
        self.indicadores = []  # Lista vacía que guardará objetos IndicadorEconomico

    def agregar_indicador(self, indicador):
        """Agrega un objeto IndicadorEconomico a la lista"""
        self.indicadores.append(indicador)  # Añade el objeto indicador a la lista interna

    def generar_grafico_interactivo(self, paises_seleccionados, start_year, end_year):
        """Genera un gráfico interactivo con los datos disponibles."""
        datos_filtrados = [i for i in self.indicadores if i.pais in paises_seleccionados and start_year <= int(i.year) <= end_year]  # Filtra indicadores por país y rango de años
        if not datos_filtrados:  # Comprueba si no hay datos tras el filtrado
            print("No hay datos para los países y años seleccionados.")
            return

        # Organizar los datos por país
        datos_por_pais = {}  # Diccionario para agrupar años y valores por país
        for i in datos_filtrados:  # Itera sobre los indicadores filtrados
            if i.pais not in datos_por_pais:  # Si el país no está en el diccionario, empieza el comando
                datos_por_pais[i.pais] = {'Año': [], 'PIB per cápita (USD)': []}  # Crea listas vacías para años y valores en un diccionario
            datos_por_pais[i.pais]['Año'].append(int(i.year))  # Añade el año (como entero) a la lista
            datos_por_pais[i.pais]['PIB per cápita (USD)'].append(i.valor)  # Añade el valor del indicador a la lista

        # Crear gráfico
        fig = go.Figure()  # Empieza una figura de Plotly
        for pais in paises_seleccionados:  # Recorre la lista de países
            if pais in datos_por_pais:  # Busca si existen datos para el país
                fig.add_trace(go.Scatter(
                    x=datos_por_pais[pais]['Año'],  # Asigna los años al eje X
                    y=datos_por_pais[pais]['PIB per cápita (USD)'],  # Asigna los valores al eje Y
                    mode='lines+markers',  # Modo de visualización
                    name=pais  # Nombre de la linea
                ))

        # Estilo del gráfico
        fig.update_layout(
            title=f"Evolución del PIB per cápita ({', '.join(paises_seleccionados)}, {start_year}-{end_year})",  # Título dinámico con países y rango de años
            xaxis_title="Año",  # Titulo del eje X
            yaxis_title="PIB per cápita (USD)",  # Titulo del eje Y
            showlegend=True  # Muestra la leyenda del gráfico
        )

        fig.show()  # Muestra el gráfico interactivo

"""
HACEMOS PRIMERO LAS CLASES YA QUE ES MEJOR Y MAS FACIL, YA
LUEGO HACEMOS LAS FUNCIONES, ADEMÁS REALIZAMOS LA INVESTIGACION EN:
https://datahelpdesk.worldbank.org/knowledgebase/articles/898581
https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?end=2024&locations=OE&start=2000&view=chart
data.worldbank.org/indicator
"""

"""TRAEMOS LOS DATOS DEL API PARA PODER HACER EL ANALISIS POR PAIS Y AÑOS DEL PIB PER CAPITA"""

def listar_paises():
    """Descarga y muestra la lista de países disponibles en la API del Banco Mundial."""
    url = "http://api.worldbank.org/v2/country?per_page=500&format=json"  # URL para obtener lista de países en formato JSON
    response = requests.get(url)  # Realiza la solicitud GET a la API del Banco Mundial para obtener los datos necesarios
    if response.status_code == 200:  # Verifica que la respuesta sea exitosa (200 = OK)
        data = response.json()  # Convierte la respuesta a JSON
        if len(data) > 1:  # Comprueba que la estructura de la respuesta contenga los datos esperados
            paises = []
            for i in data[1]:  # Itera sobre la parte de la respuesta que contiene los países
                if i['region']['value'] != 'Aggregates':  # Filtra agregados como zonas o regiones
                    paises.append({'Código': i['id'], 'Nombre': i['name']})  # Añade diccionario con código y nombre
            df = pd.DataFrame(paises)  # Crea un DataFrame de pandas con la lista de países
            print("Códigos de países disponibles:")
            print(df.to_string(index=False))  # Imprime la tabla completa sin índices
        else:
            print("Formato de respuesta inesperado.")
    else:
        print(f"Error al acceder a la API: Código {response.status_code}")

def descargar_datos_api(paises, start_year, end_year, indicador="NY.GDP.PCAP.CD"):
    """Descarga datos de PIB per cápita desde la API del Banco Mundial."""
    datos = []  # Lista para recolectar tuplas (pais, valor, año)
    for pais in paises:  # Itera por cada país solicitado
        for year in range(start_year, end_year + 1):  # Itera por cada año en el rango
            url = f"http://api.worldbank.org/v2/country/{pais}/indicator/{indicador}?date={year}&format=json"  # Construye la URL para cada país y año
            response = requests.get(url)  # Realiza la solicitud GET para obtener datos de ese país y año
            if response.status_code == 200:  # Verifica que la respuesta sea exitosa (200 = OK)
                data = response.json()  # Convierte la respuesta a JSON
                if len(data) > 1:  # Comprueba que contenga datos útiles
                    for i in data[1]:  # Itera sobre las entradas devueltas
                        value = i['value']  # Extrae el valor del indicador
                        if value is not None:  # Solo añade valores no nulos
                            datos.append((pais, value, year))  # Añade una tupla con país, valor y año
                else:
                    print(f"Datos incompletos para {pais} en {year}.")
            else:
                print(f"Error al descargar datos para {pais} en {year}: {response.status_code}")
    return datos  # Retorna la lista de datos recopilados

"""COMO LO ANTERIOR NECESITA EJECUCION ES LO QUE VAMOS A REALIZAR PROGRAMANDO
UNA FUNCION QUE NOS SIRVA PARA ELLO Y ADEMÁS INTERFAZ DE USUARIO"""

def interfaz():  # Función principal que coordina el flujo del programa
    """Ejecuta todo el flujo del programa paso a paso."""
    print("Bienvenido al Analizador de PIB per cápita del Banco Mundial.")
    print("Instrucciones:")
    print("- Ingrese códigos de países de 3 letras (ej: 'COL' para Colombia).")
    print("- Use 'list' para ver códigos disponibles.")
    print("- Escriba 'salir' en cualquier momento para finalizar.")
    print("- El programa generará un gráfico interactivo.\n")

    analizador = AnalizadorEconomico()  # Crea un objeto de AnalizadorEconomico para almacenar datos

    # Solicitar países
    print("Ingrese códigos de países (separados por coma, ej. COL,USA), 'list' para ver códigos, o 'salir' para terminar:")
    paises_input = input().strip()  # Lee la entrada del usuario y elimina espacios al inicio y final

    if paises_input.lower() == 'salir':  # Si el usuario escribe 'salir' ejecuta el comando
        print("Saliendo del programa.")
        return
    elif paises_input.lower() == 'list':  # Si el usuario solicita la lista de países
        listar_paises()  # Llama a la función listar_paises para mostrar códigos disponibles
        return

    paises = [pais.strip().upper() for pais in paises_input.split(",") if len(pais.strip()) == 3]  # Procesa la entrada: separa por comas, quita espacios y convierte a mayúsculas, filtrando códigos de 3 letras
    if not paises:  # Si no se obtuvieron códigos válidos
        print("No se ingresaron códigos válidos. Usando COL como predeterminado.")
        paises = ["COL"]  # Asigna Colombia como país predeterminado

    # Solicitar años
    start_year = input("Ingrese el año inicial (o escriba 'salir' para terminar): ").strip()  # Solicita y lee el año inicial
    if start_year.lower() == 'salir':  # Comprueba si el usuario quiere salir
        print("Saliendo del programa.")
        return

    end_year = input("Ingrese el año final (o escriba 'salir' para terminar): ").strip()  # Solicita y lee el año final
    if end_year.lower() == 'salir':  # Comprueba si el usuario quiere salir
        print("Saliendo del programa.")
        return

    # Validar si dejaron vacío
    if start_year == "" or end_year == "":  # Si alguno de los años quedó vacío
        print("No escribiste los años. Usando 2020-2022 por defecto.")
        start_year = 2020  # Año inicial por defecto
        end_year = 2022  # Año final por defecto
    else:
        # Convertir a entero
        start_year = int(start_year)  # Convierte la entrada de texto a entero para start_year
        end_year = int(end_year)  # Convierte la entrada de texto a entero para end_year
        if start_year > end_year:  # Si el año inicial es mayor que el final (rango inválido)
            print("El año inicial no puede ser mayor que el final. Usando 2020-2022 por defecto.")
            start_year = 2020  # Reinicia start_year al valor por defecto
            end_year = 2022  # Reinicia end_year al valor por defecto

    # Descargar datos
    datos_api = descargar_datos_api(paises, start_year, end_year)  # Llama a la función que descarga los datos del API para los países y años seleccionados
    if not datos_api:  # Si no se obtuvieron datos válidos
        print("No se obtuvieron datos válidos. Finalizando programa.")
        return

    # Agregar los datos
    for pais, valor, year in datos_api:
        indicador = IndicadorEconomico("PIB per cápita", pais, valor, year)  # Crea un objeto IndicadorEconomico por cada tupla
        analizador.agregar_indicador(indicador)  # Añade el indicador creado al analizador

    # Generar gráfico
    analizador.generar_grafico_interactivo(paises, start_year, end_year)

print("Programa finalizado. Gracias por usar el analizador.")


interfaz()

Programa finalizado. Gracias por usar el analizador.
Bienvenido al Analizador de PIB per cápita del Banco Mundial.
Instrucciones:
- Ingrese códigos de países de 3 letras (ej: 'COL' para Colombia).
- Use 'list' para ver códigos disponibles.
- Escriba 'salir' en cualquier momento para finalizar.
- El programa generará un gráfico interactivo.

Ingrese códigos de países (separados por coma, ej. COL,USA), 'list' para ver códigos, o 'salir' para terminar:
col, per, bol
Ingrese el año inicial (o escriba 'salir' para terminar): 2000
Ingrese el año final (o escriba 'salir' para terminar): 2002


In [ ]:
interfaz()

Bienvenido al Analizador de PIB per cápita del Banco Mundial.
Instrucciones:
- Ingrese códigos de países de 3 letras (ej: 'COL' para Colombia).
- Use 'list' para ver códigos disponibles.
- Escriba 'salir' en cualquier momento para finalizar.
- El programa generará un gráfico interactivo.

Ingrese códigos de países (separados por coma, ej. COL,USA), 'list' para ver códigos, o 'salir' para terminar:
LIST
Códigos de países disponibles:
Código                         Nombre
   ABW                          Aruba
   AFG                    Afghanistan
   AGO                         Angola
   ALB                        Albania
   AND                        Andorra
   ARE           United Arab Emirates
   ARG                      Argentina
   ARM                        Armenia
   ASM                 American Samoa
   ATG            Antigua and Barbuda
   AUS                      Australia
   AUT                        Austria
   AZE                     Azerbaijan
   BDI                        B